In [ ]:
!pip install snowflake-connector-python pandas requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 2.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of pyopenssl to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: cryptography
    Found existing installation: cryptography 43.0.3
    Uninstalling cryptography-43.0.3:
      Successfully uninstalled cryptography-43.0.3
  Attempting uninstall: pyOpenSSL
    Found ex

In [ ]:
"""
Restaurant Reviews Collection from Yelp and Google Places APIs
Fetches reviews for all restaurants in GOLD_RESTAURANTS_MASTER
"""

import requests
import pandas as pd
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
from datetime import datetime
import time

# =====================================================
# CONFIGURATION
# =====================================================

SNOWFLAKE_CONFIG = {
    'user': '',
    'password':'',
    'account': '',
    'warehouse': '',
    'database': '',
    'schema': '',
    'role': ''
}

In [ ]:
# API Keys - UPDATE THESE
YELP_API_KEY = ''
GOOGLE_API_KEY = ''

In [ ]:
YELP_API_KEYS = [
    '',  # For Central Boston
    '',  # For Cambridge/Somerville
    ''   # For Outer Boston
]

SNOWFLAKE_CONFIG = {
    'user': '',
    'password': '',
    'account': '',
    'warehouse': '',
    'database': '',
    'schema': '',
    'role': ''
}

In [ ]:
# =====================================================
# FUNCTIONS
# =====================================================

def get_restaurants_from_snowflake(conn):
    """Fetch all restaurants from Gold table"""
    query = """
    SELECT
        RESTAURANT_ID,
        GOOGLE_PLACE_ID,
        RESTAURANT_NAME,
        CITY,
        NEIGHBORHOOD
    FROM LOCEATS_DB.DBT_SKASIREDDY_MARTS.GOLD_RESTAURANTS_MASTER
    WHERE GOOGLE_PLACE_ID IS NOT NULL
    """

    print("📥 Fetching restaurants from Snowflake...")
    cursor = conn.cursor()
    cursor.execute(query)
    restaurants = cursor.fetchall()
    print(f"✅ Loaded {len(restaurants)} restaurants\n")
    return restaurants


def get_yelp_reviews(business_id):
    """Fetch up to 3 reviews from Yelp API"""
    url = f'https://api.yelp.com/v3/businesses/{business_id}/reviews'
    headers = {'Authorization': f'Bearer {YELP_API_KEY}'}

    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            data = response.json()
            return data.get('reviews', [])
        elif response.status_code == 404:
            return []  # Business not found
        else:
            print(f"    Yelp API error: {response.status_code}")
            return []
    except Exception as e:
        print(f"    Yelp exception: {str(e)}")
        return []


def get_google_reviews(place_id):
    """Fetch reviews from Google Places API"""
    url = 'https://maps.googleapis.com/maps/api/place/details/json'
    params = {
        'place_id': place_id,
        'fields': 'reviews',
        'key': GOOGLE_API_KEY
    }

    try:
        response = requests.get(url, params=params, timeout=10)
        if response.status_code == 200:
            data = response.json()
            if data.get('status') == 'OK':
                return data.get('result', {}).get('reviews', [])
            elif data.get('status') == 'NOT_FOUND':
                return []
            else:
                print(f"    Google API status: {data.get('status')}")
                return []
        else:
            print(f"    Google API error: {response.status_code}")
            return []
    except Exception as e:
        print(f"    Google exception: {str(e)}")
        return []


def collect_all_reviews(restaurants):
    """Collect reviews from both Yelp and Google for all restaurants"""

    print("Starting review collection...\n")
    print(f"{'='*70}")

    all_reviews = []
    total_yelp_reviews = 0
    total_google_reviews = 0

    for idx, (restaurant_id, place_id, name, city, neighborhood) in enumerate(restaurants, 1):
        print(f"\n[{idx}/{len(restaurants)}] {name}")
        print(f"    Location: {neighborhood or city}")

        # Fetch Yelp reviews
        yelp_reviews = get_yelp_reviews(restaurant_id)
        for review in yelp_reviews:
            all_reviews.append({
                'restaurant_id': restaurant_id,
                'restaurant_name': name,
                'source': 'yelp',
                'review_id': f"yelp_{restaurant_id}_{review.get('id', '')}",
                'review_text': review.get('text', ''),
                'rating': float(review.get('rating', 0)),
                'review_date': review.get('time_created', ''),
                'user_name': review.get('user', {}).get('name', ''),
                'collected_at': datetime.now()
            })
        total_yelp_reviews += len(yelp_reviews)
        print(f"    ✓ Yelp: {len(yelp_reviews)} reviews")

        # Fetch Google reviews
        google_reviews = get_google_reviews(place_id)
        for review in google_reviews:
            try:
                review_timestamp = review.get('time', 0)
                review_date = datetime.fromtimestamp(review_timestamp).strftime('%Y-%m-%d %H:%M:%S')
            except:
                review_date = None

            all_reviews.append({
                'restaurant_id': restaurant_id,
                'restaurant_name': name,
                'source': 'google',
                'review_id': f"google_{place_id}_{review.get('time', '')}",
                'review_text': review.get('text', ''),
                'rating': float(review.get('rating', 0)),
                'review_date': review_date,
                'user_name': review.get('author_name', ''),
                'collected_at': datetime.now()
            })
        total_google_reviews += len(google_reviews)
        print(f"    ✓ Google: {len(google_reviews)} reviews")

        # Rate limiting to avoid API throttling
        time.sleep(0.3)

    print(f"\n{'='*70}")
    print(f"COLLECTION COMPLETE")
    print(f"{'='*70}")
    print(f"Total Yelp reviews: {total_yelp_reviews}")
    print(f"Total Google reviews: {total_google_reviews}")
    print(f"Total reviews collected: {len(all_reviews)}")

    return all_reviews


# =====================================================
# MAIN
# =====================================================

def main():
    print("🚀 Restaurant Reviews Collection")
    print("="*70)
    print("Sources: Yelp API + Google Places API")
    print("Target: All restaurants in GOLD_RESTAURANTS_MASTER\n")

    start_time = datetime.now()

    # Connect to Snowflake
    print("Connecting to Snowflake...")
    conn = snowflake.connector.connect(**SNOWFLAKE_CONFIG)
    print("Connected!\n")

    # Get restaurants
    restaurants = get_restaurants_from_snowflake(conn)

    # Collect reviews
    reviews = collect_all_reviews(restaurants)

    if not reviews:
        print("No reviews collected!")
        conn.close()
        return

    # Convert to DataFrame
    df = pd.DataFrame(reviews)

    # Show statistics
    print(f"\n📈 STATISTICS")
    print(f"{'='*70}")
    print(f"Unique restaurants with reviews: {df['restaurant_id'].nunique()}")
    print(f"Average reviews per restaurant: {len(df) / df['restaurant_id'].nunique():.1f}")
    print(f"Average rating (Yelp): {df[df['source']=='yelp']['rating'].mean():.2f}")
    print(f"Average rating (Google): {df[df['source']=='google']['rating'].mean():.2f}")

    # Top reviewed restaurants
    print(f"\n🔥 TOP 10 MOST REVIEWED RESTAURANTS:")
    top_reviewed = df.groupby('restaurant_name').size().sort_values(ascending=False).head(10)
    for name, count in top_reviewed.items():
        print(f"   {name}: {count} reviews")

    # Prepare for Snowflake
    df.columns = df.columns.str.upper()

    # Create table and load data
    print(f"Loading to Snowflake...")

    cursor = conn.cursor()
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS BRONZE_RESTAURANT_REVIEWS (
        REVIEW_ID VARCHAR(255) PRIMARY KEY,
        RESTAURANT_ID VARCHAR(50),
        RESTAURANT_NAME VARCHAR(500),
        SOURCE VARCHAR(10),
        REVIEW_TEXT VARCHAR(16777216),
        RATING FLOAT,
        REVIEW_DATE VARCHAR(50),
        USER_NAME VARCHAR(255),
        COLLECTED_AT TIMESTAMP_NTZ
    )
    """)

    success, nchunks, nrows, _ = write_pandas(
        conn,
        df,
        'BRONZE_RESTAURANT_REVIEWS',
        database='LOCEATS_DB',
        schema='DBT_SKASIREDDY_MARTS',
        auto_create_table=False,
        quote_identifiers=False
    )

    if success:
        print(f"Successfully loaded {nrows} reviews to Snowflake!")
    else:
        print("Failed to load data")

    # Verification query
    print(f"Verifying data in Snowflake...")
    verify_query = """
    SELECT
        SOURCE,
        COUNT(*) as review_count,
        AVG(RATING) as avg_rating
    FROM BRONZE_RESTAURANT_REVIEWS
    GROUP BY SOURCE
    """
    verification = pd.read_sql(verify_query, conn)
    print("\nReviews by source:")
    print(verification.to_string(index=False))

    conn.close()

    duration = (datetime.now() - start_time).total_seconds() / 60
    print(f"\n{'='*70}")
    print(f"Total time: {duration:.1f} minutes")
    print(f"Data location: LOCEATS_DB.DBT_SKASIREDDY_MARTS.BRONZE_RESTAURANT_REVIEWS")
    print(f"{'='*70}")


if __name__ == "__main__":
    main()

Streaming output truncated to the last 5000 lines.
    Location: North End
    ✓ Yelp: 0 reviews
    ✓ Google: 5 reviews

[2001/2994] Upper Crust Pizzeria
    Location: Coolidge Corner
    ✓ Yelp: 0 reviews
    ✓ Google: 5 reviews

[2002/2994] Dirty Water Dough
    Location: Back Bay
    ✓ Yelp: 0 reviews
    ✓ Google: 5 reviews

[2003/2994] Scholars American Bistro & Cocktail Club
    Location: Financial District
    ✓ Yelp: 0 reviews
    ✓ Google: 5 reviews

[2004/2994] The Diner At 11 North Beacon
    Location: Brighton
    ✓ Yelp: 0 reviews
    ✓ Google: 5 reviews

[2005/2994] Jugos Supremo
    Location: Nubian Square
    ✓ Yelp: 0 reviews
    ✓ Google: 5 reviews

[2006/2994] Harvard Gardens
    Location: Downtown Boston
    ✓ Yelp: 0 reviews
    ✓ Google: 5 reviews

[2007/2994] Naco Taco
    Location: Central Square Cambridge
    ✓ Yelp: 0 reviews
    ✓ Google: 5 reviews

[2008/2994] Tango Mango
    Location: Newton Centre
    ✓ Yelp: 0 reviews
    ✓ Google: 5 reviews

[2009/2994]

ProgrammingError: 000904 (42000): SQL compilation error: error line 1 at position 100
invalid identifier 'RESTAURANT_NAME'